In [2]:
import pandas as pd
from cleaning_function import *
data = pd.read_csv('cvrm_master.csv')

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_32252/187753911.py:3: DtypeWarning: Columns (1697,1698) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cvrm_master.csv')


In [4]:
data = drop_null_columns(data, 0.6)
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import warnings

import warnings
warnings.filterwarnings("ignore")

def gmm_imputation(df, n_components=3):
    """Impute missing values using Gaussian Mixture Model (GMM)."""
    df_imputed = df.copy()
    for col in df.columns:
        if df[col].isna().any(): 
            valid_data = df[[col]].dropna()
            if valid_data.nunique().values[0] > 1:  
                gmm = GaussianMixture(n_components=n_components, random_state=42)
                gmm.fit(valid_data)
                missing_indices = df[col].isna()
                df_imputed.loc[missing_indices, col] = gmm.sample(missing_indices.sum())[0].flatten()
            else:
                df_imputed[col].fillna(valid_data.mean().values[0], inplace=True)
    return df_imputed


data = drop_null_columns(data, 0.6)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
numerical_cols = data.select_dtypes(include=['number']).columns.tolist()
data_numeric = data[numerical_cols]

kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_rmse = []

for fold, (train_idx, test_idx) in enumerate(kf.split(data_numeric)):
    print(f"Starting Fold {fold + 1}")


    train_data, test_data = data_numeric.iloc[train_idx], data_numeric.iloc[test_idx]

    train_imputed = gmm_imputation(train_data)
    test_imputed = gmm_imputation(test_data)
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_imputed)
    test_scaled = scaler.transform(test_imputed)

    pca = PCA(n_components=min(20, train_scaled.shape[1]))  
    train_reduced = pca.fit_transform(train_scaled)
    test_reduced = pca.transform(test_scaled)

    model = Ridge(alpha=1.0)
    model.fit(train_reduced, train_scaled)  
    test_predictions = model.predict(test_reduced)
    mse = mean_squared_error(test_scaled, test_predictions)
    rmse = np.sqrt(mse)
    all_rmse.append(rmse)
    
    print(f"Fold {fold + 1} RMSE: {rmse}")

print(f"Average RMSE across all folds: {sum(all_rmse) / len(all_rmse)}")


Starting Fold 1
Fold 1 RMSE: 0.6418471857825581
Starting Fold 2
Fold 2 RMSE: 0.6433850814249468
Starting Fold 3
Fold 3 RMSE: 22.25641970966385
Starting Fold 4
Fold 4 RMSE: 0.6353782986734781
Starting Fold 5
Fold 5 RMSE: 0.5898924670172181
Average RMSE across all folds: 4.9533845485124095
